In [13]:
import pandas as pd
import os

## MAPPINGS
- *Phase 1 (f1): Utgångsposition -> 11 aspects*
- *Phase 2 (f2): Rörelsestart -> 1 aspect*
- *Phase 3 (f3): Rörelseutförande onwards -> 12 aspects*

In [14]:
INPUT_FILE = '../data/original/Final_IRAF_SAT_Bedömare sep och gemensamt_AL.xlsx'
OUTPUT_FILE = '../data/processed/OMA_Score/OMA_Score.csv'

PHASE_MAPPING = {
    "Utgångsposition": "f1",
    "Rörelsestart": "f2",
    "Rörelseutförande": "f3"
}

MOVEMENT_MAPPING = {
    "Sittande till stående": "2a_1",
    "Stående till sittande": "2a_2"
}

In [ ]:
def ensure_output_directory(file_path):
    directory = os.path.dirname(file_path)
    if directory and not os.path.exists(directory):
        os.makedirs(directory)

def parse_time(raw_tid):
    if pd.isna(raw_tid):
        return "", ""
    tid_str = str(raw_tid).strip()
    if '-' in tid_str:
        parts = tid_str.split('-')
        return parts[0].strip(), parts[1].strip()
    return tid_str, ""

In [16]:
def extract_oma_data(file_path, phase_mapping, movement_mapping):
    xl = pd.ExcelFile(file_path)
    all_extracted_rows = []
    
    for sheet_name in xl.sheet_names:
        movement_id = movement_mapping.get(sheet_name)
        if movement_id is None:
            continue
        
        df = pd.read_excel(file_path, sheet_name=sheet_name)
        
        current_person_id = None
        current_rorelse_id = movement_id
        current_phase_id = None
        current_start_tid = None
        current_end_tid = None
        current_aspekt_counter = 0
        seen_phases = set()
        phase_start_times = {}
        extracted_rows = []
        
        h0 = str(df.columns[0]).strip()
        if h0.lower().startswith("fp") and "_" in h0:
            parts = h0.split("_", 1)
            current_person_id = parts[0].replace(" ", "").lower()

        for idx, row in df.iterrows():
            raw = str(row.iloc[0]) if pd.notna(row.iloc[0]) else ""
            col0 = raw.replace("\xa0", " ").strip()
            raw_tid = row['Tid i film'] if 'Tid i film' in df.columns else None

            if not col0:
                continue

            if col0.lower().startswith("fp") and "_" in col0:
                parts = col0.split("_", 1)
                current_person_id = parts[0].replace(" ", "").lower()
                current_phase_id = None
                current_start_tid = None
                current_end_tid = None
                current_aspekt_counter = 0
                seen_phases = set()
                phase_start_times = {}
                continue

            new_phase_id = phase_mapping.get(col0)
            
            if new_phase_id:
                if pd.notna(raw_tid): 
                    s, e = parse_time(raw_tid)
                    
                    if new_phase_id not in phase_start_times:
                        phase_start_times[new_phase_id] = s
                    
                    if current_phase_id and current_phase_id != new_phase_id:
                        for row_data in extracted_rows:
                            if (row_data['PersonId'] == current_person_id and 
                                row_data['PhaseId'] == current_phase_id and
                                row_data['SlutTid'] == ""):
                                row_data['SlutTid'] = s
                    
                    if new_phase_id not in seen_phases:
                        current_start_tid = s
                        current_end_tid = e if e else ""
                        current_phase_id = new_phase_id
                        current_aspekt_counter = 0  
                        seen_phases.add(new_phase_id)
                    elif e:
                        current_end_tid = e
                
                continue 
                 
            if current_person_id and current_phase_id:
                current_aspekt_counter += 1
                if pd.notna(raw_tid):
                    s, e = parse_time(raw_tid)
                    row_start, row_end = s, e if e else ""
                else:
                    row_start = current_start_tid if current_start_tid else ""
                    if current_phase_id == "f3":
                        row_end = current_start_tid if current_start_tid else ""
                    else:
                        row_end = current_end_tid if current_end_tid else ""
                      
                score = row.get('Gem_Avvikelse (0-2)', "")
                if pd.isna(score): 
                    score = ""
                 
                extracted_rows.append({
                    'PersonId': current_person_id,
                    'RörelseId': current_rorelse_id,
                    'PhaseId': current_phase_id,
                    'StartTid': row_start,
                    'SlutTid': row_end, 
                    'AspektId': current_aspekt_counter,
                    'Gem_Avvikelse': score,
                    '_phase_end_tid': current_end_tid
                })
        
        all_extracted_rows.extend(extracted_rows)
    
    result_df = pd.DataFrame(all_extracted_rows)
    
    for person_id in result_df['PersonId'].unique():
        for mov_id in result_df['RörelseId'].unique():
            mask = (result_df['PersonId'] == person_id) & (result_df['RörelseId'] == mov_id) & (result_df['PhaseId'] == 'f3')
            f3_indices = result_df[mask].index
            if len(f3_indices) > 0:
                last_idx = f3_indices[-1]
                end_time = result_df.loc[last_idx, '_phase_end_tid']
                if end_time:
                    result_df.loc[last_idx, 'SlutTid'] = end_time
    
    result_df = result_df.drop(columns=['_phase_end_tid'])
    return result_df

In [17]:
ensure_output_directory(OUTPUT_FILE)

df = extract_oma_data(INPUT_FILE, PHASE_MAPPING, MOVEMENT_MAPPING)
df = df[['PersonId', 'RörelseId', 'PhaseId', 'StartTid', 'SlutTid', 'AspektId', 'Gem_Avvikelse']]

print("Processing complete. First 10 rows:")
print(df.head(10))

df.to_csv(OUTPUT_FILE, index=False, encoding='utf-8-sig')
print(f"\nData saved to: {OUTPUT_FILE}")

Processing complete. First 10 rows:
  PersonId RörelseId PhaseId StartTid SlutTid  AspektId Gem_Avvikelse
0      fp1      2a_1      f1    00.00   00.01         1             0
1      fp1      2a_1      f1    00.00   00.01         2             0
2      fp1      2a_1      f1    00.00   00.01         3             0
3      fp1      2a_1      f1    00.00   00.01         4             0
4      fp1      2a_1      f1    00.00   00.01         5              
5      fp1      2a_1      f1    00.00   00.01         6              
6      fp1      2a_1      f1    00.00   00.01         7              
7      fp1      2a_1      f1    00.00   00.01         8             0
8      fp1      2a_1      f1    00.00   00.01         9             1
9      fp1      2a_1      f1    00.00   00.01        10             0

Data saved to: ../data/processed/OMA_Score/OMA_Score.csv


In [18]:
def time_to_seconds(time_str):
    if pd.isna(time_str) or time_str == "":
        return None
    try:
        parts = str(time_str).split('.')
        if len(parts) == 2:
            return int(parts[0]) * 60 + int(parts[1])
        return int(parts[0]) 
    except (ValueError, TypeError):
        return None

In [19]:
print("1. Basic Data Overview ")
print(f"Shape: {df.shape[0]} rows, {df.shape[1]} columns")
print("\nFirst 5 rows:")
print(df.head())

1. Basic Data Overview 
Shape: 245 rows, 7 columns

First 5 rows:
  PersonId RörelseId PhaseId StartTid SlutTid  AspektId Gem_Avvikelse
0      fp1      2a_1      f1    00.00   00.01         1             0
1      fp1      2a_1      f1    00.00   00.01         2             0
2      fp1      2a_1      f1    00.00   00.01         3             0
3      fp1      2a_1      f1    00.00   00.01         4             0
4      fp1      2a_1      f1    00.00   00.01         5              


In [20]:
print("\n2. Completeness Check")
try:
    source_df = pd.read_excel(INPUT_FILE, sheet_name=0)
    source_persons = source_df.iloc[:, 0].dropna()
    source_unique_persons = source_persons[source_persons.str.startswith("FP", na=False)].nunique()
    output_unique_persons = df['PersonId'].nunique()
    
    print(f"Unique persons in source: {source_unique_persons}")
    print(f"Unique persons in output: {output_unique_persons}")
    print("Person count matches!" if source_unique_persons == output_unique_persons else "Person count MISMATCH!")
except Exception as e:
    print(f"Could not compare with source file: {e}")

print(f"\nUnique movements: {df['RörelseId'].value_counts().to_dict()}")
print(f"Unique phases: {df['PhaseId'].value_counts().to_dict()}")
print(f"Total aspects (rows): {len(df)}")


2. Completeness Check
Unique persons in source: 0
Unique persons in output: 5
Person count MISMATCH!

Unique movements: {'2a_2': 125, '2a_1': 120}
Unique phases: {'f1': 130, 'f3': 105, 'f2': 10}
Total aspects (rows): 245


In [21]:
print("\n3. Mapping Validation ")
print(f"Phase IDs found: {sorted(df['PhaseId'].unique())}")
print(f"Movement IDs found: {sorted(df['RörelseId'].unique())}")


3. Mapping Validation 
Phase IDs found: ['f1', 'f2', 'f3']
Movement IDs found: ['2a_1', '2a_2']


In [22]:
print("\n 4. Time Consistency Checks")

df['StartTid_sec'] = df['StartTid'].apply(time_to_seconds)
df['SlutTid_sec'] = df['SlutTid'].apply(time_to_seconds)

invalid_times = df[(df['StartTid_sec'] > df['SlutTid_sec']) & (df['SlutTid_sec'].notna())]
if not invalid_times.empty:
    print("Found rows with invalid time ranges (start > end):")
    print(invalid_times[['PersonId', 'PhaseId', 'StartTid', 'SlutTid']])
else:
    print("All time ranges are valid (start ≤ end)")


 4. Time Consistency Checks
All time ranges are valid (start ≤ end)


In [23]:
print("\n 5. Data Integrity and Scores")

missing = df[['PersonId', 'RörelseId', 'PhaseId', 'AspektId', 'Gem_Avvikelse']].isnull().sum()
print("Missing values:")
for col, count in missing.items():
    print(f"  {col}: {count}")

print("\nScore distribution:")
print(df['Gem_Avvikelse'].astype(str).value_counts().sort_index())


 5. Data Integrity and Scores
Missing values:
  PersonId: 0
  RörelseId: 0
  PhaseId: 0
  AspektId: 0
  Gem_Avvikelse: 0

Score distribution:
Gem_Avvikelse
     71
0    75
1    84
2    15
Name: count, dtype: int64


In [24]:
print("\n 6. Data Structure Example")
example_person = df['PersonId'].iloc[0]
print(f"Showing data structure for: {example_person}\n")

person_data = df[df['PersonId'] == example_person]
for movement, movement_group in person_data.groupby('RörelseId'):
    print(f"Movement: {movement}")
    for phase, phase_group in movement_group.groupby('PhaseId'):
        print(f"  Phase: {phase}")
        aspects = phase_group[['AspektId', 'StartTid', 'SlutTid', 'Gem_Avvikelse']].sort_values('AspektId')
        print(aspects.to_string(index=False))
    print("-" * 20)


 6. Data Structure Example
Showing data structure for: fp1

Movement: 2a_1
  Phase: f1
 AspektId StartTid SlutTid Gem_Avvikelse
        1    00.00   00.01             0
        2    00.00   00.01             0
        3    00.00   00.01             0
        4    00.00   00.01             0
        5    00.00   00.01              
        6    00.00   00.01              
        7    00.00   00.01              
        8    00.00   00.01             0
        9    00.00   00.01             1
       10    00.00   00.01             0
       11    00.00   00.01             0
  Phase: f2
 AspektId StartTid SlutTid Gem_Avvikelse
        1    00.01   00.01             0
  Phase: f3
 AspektId StartTid SlutTid Gem_Avvikelse
        1    00.01   00.01             0
        2    00.01   00.01              
        3    00.01   00.01             0
        4    00.01   00.01             1
        5    00.01   00.01             0
        6    00.01   00.01             0
        7    00.01   00.01 